In [ ]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

import pandas as pd
import requests
import json
import urllib3

In [ ]:
# Carrega as variáveis do arquivo .env
load_dotenv()
database_url = os.getenv('DATABASE_URL')

# Faz conexão com Atlas
cliente = MongoClient(database_url)
db = cliente['projeto_fecomercio']

In [ ]:
# Desabilitar a verificação de certificado SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://api-comexstat.mdic.gov.br/cities"

headers = {
    "Content-Type": "application/json"
}

filtrosStateExport = { "flow": "export",
            "monthDetail": True,
            "period": { "from": "2020-01", "to": "2024-12" },
            "filters": [ { "filter": "country", "values": [ 105] }],
            "details": [ "state"],
            "metrics": [ "metricFOB", "metricKG" ] }

filtrosStateImport = { "flow": "import",
            "monthDetail": True,
            "period": { "from": "2020-01", "to": "2024-12" },
            "filters": [ { "filter": "country", "values": [ 105] }],
            "details": [ "state"],
            "metrics": [ "metricFOB", "metricKG" ] }

filtrosCityExport = { "flow": "export",
            "monthDetail": True,
            "period": { "from": "2020-01", "to": "2024-12" },
            "filters": [ { "filter": "country", "values": [ 105] }],
            "details": [ "city"],
            "metrics": [ "metricFOB", "metricKG" ] }

filtrosCityImport = { "flow": "import",
            "monthDetail": True,
            "period": { "from": "2020-01", "to": "2024-12" },
            "filters": [ { "filter": "country", "values": [ 105] }],
            "details": [ "city"],
            "metrics": [ "metricFOB", "metricKG" ] }

# Fazendo a requisição POST com verify=False
responseStateExport = requests.post(url, json=filtrosStateExport, headers=headers, verify=False)
responseStateImport = requests.post(url, json=filtrosStateImport, headers=headers, verify=False)
responseCityExport = requests.post(url, json=filtrosCityExport, headers=headers, verify=False)
responseCityImport = requests.post(url, json=filtrosCityImport, headers=headers, verify=False)

In [ ]:
# Checando se a requisição da exportação de cidade foi bem-sucedida
print('responseCityExport: ', end='')
if responseCityExport.status_code == 200:
    dadosCityExport = responseCityExport.json()  # Converte a resposta em JSON
    print('ok')
else:
    print("Erro:", responseCityExport.status_code, responseCityExport.text)

# Checando se a requisição da importação de cidade foi bem-sucedida
print('responseCityImport: ', end='')
if responseCityImport.status_code == 200:
    dadosCityImport = responseCityImport.json()  # Converte a resposta em JSON
    print('ok')
else:
    print("Erro:", responseCityImport.status_code, responseCityImport.text)

# Checando se a requisição da exportação de estado foi bem-sucedida
print('responseStateExport: ', end='')
if responseStateExport.status_code == 200:
    dadosStateExport = responseStateExport.json()  # Converte a resposta em JSON
    print('ok')
else:
    print("Erro:", responseStateExport.status_code, responseStateExport.text)

# Checando se a requisição da importação de estado foi bem-sucedida
print('responseStateImport: ', end='')
if responseStateImport.status_code == 200:
    dadosStateImport = responseStateImport.json()  # Converte a resposta em JSON
    print('ok')
else:
    print("Erro:", responseStateImport.status_code, responseStateImport.text)

In [ ]:
# Extrair a lista de dados da importação de estado
data_list = dadosStateImport['data']['list']

# Transformar em DataFrame
df_importState = pd.DataFrame(data_list)

# Exibir o DataFrame
df_importState.sample(20)

#df_importState.shape #retornar o numero de registros e variaveis

#df_importState["state"].value_counts()

In [ ]:
# Extrair a lista de dados da exportação de estado
data_list = dadosStateExport['data']['list']

# Transformar em DataFrame
df_exportState = pd.DataFrame(data_list)

# Exibir o DataFrame
df_exportState.sample(20)

#df_exportState.shape #retornar o numero de registros e variaveis

#df_exportState["state"].value_counts()

In [ ]:
# Extrair a lista de dados da importação de cidade
data_list = dadosCityImport['data']['list']

# Transformar em DataFrame
df_importCountry = pd.DataFrame(data_list)

# Exibir o DataFrame
df_importCountry.sample(20)

#df_importCountry.shape #retornar o numero de registros e variaveis

#df_importCountry["state"].value_counts()

In [ ]:
# Extrair a lista de dados da exportação de cidade
data_list = dadosCityExport['data']['list']

# Transformar em DataFrame
df_exportCountry = pd.DataFrame(data_list)

# Exibir o DataFrame
df_exportCountry.sample(20)

#df_exportCountry.shape #retornar o numero de registros e variaveis

#df_exportCountry["state"].value_counts()

### manda para o mongo

In [ ]:
collection_importState = db['comexstatImportState']
collection_importState.insert_many(df_importState.to_dict('records'))

collection_exportState = db['comexstatExportState']
collection_exportState.insert_many(df_exportState.to_dict('records'))

collection_importCountry = db['comexstatImportCountry']
collection_importCountry.insert_many(df_importCountry.to_dict('records'))

collection_exportCountry = db['comexstatExportCountry']
collection_exportCountry.insert_many(df_exportCountry.to_dict('records'))